In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import decomposition
from scipy.stats import boxcox

In [2]:
Glasgow_DataZone = pd.read_csv("E:/文档/Urban Analytics/Dissertation/Data/DataZone2020.txt")
Glasgow_DataZone = Glasgow_DataZone['DataZone']
Glasgow_DataZone

0      S01009758
1      S01009759
2      S01009760
3      S01009761
4      S01009762
         ...    
741    S01010499
742    S01010500
743    S01010501
744    S01010502
745    S01010503
Name: DataZone, Length: 746, dtype: object

## Gentrification score calculation 2016

In [3]:
houseprice_16 = pd.read_csv('E:/文档/Urban Analytics/Dissertation/Data/houseprice_16.csv')
SIMD2016 = pd.read_csv("E:/文档/Urban Analytics/Dissertation/Data/simd2016_withgeog/simd2016_withgeog/simd2016_withinds.csv")
SIMD2016

,Data_Zone,Intermediate_Zone,Council_area,Total_population,Working_age_population_Revised,Overal_SIMD16_Rank,SIMD_2016_Percentile,SIMD_2016_Vigintile,SIMD_2016_Decile,SIMD_2016_Quintile,...,drive_retail,drive_secondary,PT_GP,PT_Post,PT_retail,crime_rate,overcrowded_count,nocentralheat_count,overcrowded_rate,nocentralheat_rate
0,S01006506,Culter,Aberdeen_City,904.0,605.0,5272.0,76.0,16.0,8.0,4.0,...,1.5,10.8,8.4,6.0,5.7,89,87.0,10.0,10%,1%
1,S01006507,Culter,Aberdeen_City,830.0,491.0,4838.0,70.0,14.0,7.0,4.0,...,2.7,11.5,8.3,7.3,6.8,48,85.0,4.0,10%,0%
2,S01006508,Culter,Aberdeen_City,694.0,519.0,6321.0,91.0,19.0,10.0,5.0,...,1.9,11.5,7.9,5.8,5.3,58,31.0,8.0,5%,1%
3,S01006509,Culter,Aberdeen_City,573.0,354.0,5363.0,77.0,16.0,8.0,4.0,...,2.2,10.8,7.4,8.3,8.4,0,42.0,6.0,7%,1%
4,S01006510,Culter,Aberdeen_City,676.0,414.0,4049.0,59.0,12.0,6.0,3.0,...,1.9,10.6,5.1,6.6,6.6,178,50.0,7.0,9%,1%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6974,S01013480,Broxburn_East,West_Lothian,663.0,382.0,2721.0,39.0,8.0,4.0,2.0,...,2.1,6.3,8.2,5.9,6.2,167,107.0,1.0,16%,0%
6975,S01013481,Broxburn_East,West_Lothian,465.0,296.0,4311.0,62.0,13.0,7.0,4.0,...,2.4,6.2,11.5,8.8,9.4,302,67.0,2.0,15%,0%
6976,# Error in the working age population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6977,# In a previous version of this file we includ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
Glasgow_SIMD2016 = pd.merge(Glasgow_DataZone, SIMD2016, how = 'inner', left_on='DataZone', right_on='Data_Zone')
Glasgow_SIMD2016 = Glasgow_SIMD2016.dropna(how='any', subset=['Income_rate','Employment_rate','HESA'])
Glasgow_SIMD2016 = Glasgow_SIMD2016.drop(Glasgow_SIMD2016[Glasgow_SIMD2016['Income_rate'] == '#VALUE!'].index)
Glasgow_SIMD2016['Income_rate'] = Glasgow_SIMD2016['Income_rate'].str.strip("%").astype('float')/100
Glasgow_SIMD2016['Employment_rate'] = Glasgow_SIMD2016['Employment_rate'].str.strip("%").astype('float')/100
Glasgow_SIMD2016['HESA'] = Glasgow_SIMD2016['HESA'].str.strip("%").astype('float')/100
Glasgow_SIMD2016['income_rate_reverse'] = Glasgow_SIMD2016['Income_rate'].map(lambda x: 1 - x)
Glasgow_SIMD2016['employment_rate_reverse'] = Glasgow_SIMD2016['Employment_rate'].map(lambda x: 1 - x)

In [5]:
Matrix2016 = Glasgow_SIMD2016.loc[:,['Data_Zone','income_rate_reverse', 'employment_rate_reverse','HESA']]
Matrix2016 = pd.merge(Matrix2016, houseprice_16, left_on = 'Data_Zone', right_on = 'DataZone', how = 'inner')
Matrix2016 = Matrix2016.dropna(how = 'any', subset = ['50%']) # there is a NaN value in 282 row
Matrix2016 = Matrix2016.reset_index()
Matrix2016_pca = Matrix2016.loc[:,['50%','income_rate_reverse', 'employment_rate_reverse','HESA']]
Matrix2016_pca

,50%,income_rate_reverse,employment_rate_reverse,HESA
0,57916.666667,0.93,0.95,0.12
1,47500.000000,0.62,0.73,0.04
2,45416.666667,0.74,0.76,0.11
3,62997.500000,0.89,0.92,0.12
4,24749.375000,0.70,0.80,0.02
...,...,...,...,...
712,21248.750000,0.58,0.64,0.02
713,56250.000000,0.66,0.73,0.03
714,42497.500000,0.75,0.80,0.04
715,46957.500000,0.80,0.85,0.03


In [6]:
pca = decomposition.PCA(n_components=1)             # Only need 1st Principal Component
pca.fit(Matrix2016_pca)                                   #  Train model on data
Matrix2016_T = pd.DataFrame(pca.transform(Matrix2016_pca))  #  Transform data using model

print("The amount of explained variance of the SES score is: {0:6.5f}".format(pca.explained_variance_ratio_[0]))

The amount of explained variance of the SES score is: 1.00000


Explained variance is 1. Is it because house price column ('50%') dominates the matrix?

In [7]:
Matrix2016_T

,0
0,6703.500931
1,-3713.165738
2,-5796.499070
3,11784.334264
4,-26463.790737
...,...
712,-29964.415738
713,5036.834262
714,-8715.665737
715,-4255.665737


In [8]:
Matrix2016_result = pd.concat([Matrix2016,Matrix2016_T],axis = 1)

In [9]:
Matrix2016_result

,index,Data_Zone,income_rate_reverse,employment_rate_reverse,HESA,DataZone,50%,0
0,0,S01009758,0.93,0.95,0.12,S01009758,57916.666667,6703.500931
1,1,S01009759,0.62,0.73,0.04,S01009759,47500.000000,-3713.165738
2,2,S01009760,0.74,0.76,0.11,S01009760,45416.666667,-5796.499070
3,3,S01009761,0.89,0.92,0.12,S01009761,62997.500000,11784.334264
4,4,S01009762,0.70,0.80,0.02,S01009762,24749.375000,-26463.790737
...,...,...,...,...,...,...,...,...
712,713,S01010499,0.58,0.64,0.02,S01010499,21248.750000,-29964.415738
713,714,S01010500,0.66,0.73,0.03,S01010500,56250.000000,5036.834262
714,715,S01010501,0.75,0.80,0.04,S01010501,42497.500000,-8715.665737
715,716,S01010502,0.80,0.85,0.03,S01010502,46957.500000,-4255.665737


## Gentrification score calculation 2020

In [10]:
houseprice_20 = pd.read_csv('E:/文档/Urban Analytics/Dissertation/Data/houseprice_20.csv')
SIMD2020 = pd.read_csv("E:/文档/Urban Analytics/Dissertation/Data/simd2020_withgeog/simd2020_withgeog/simd2020_withinds.csv")
SIMD2020

,Data_Zone,Intermediate_Zone,Council_area,Total_population,Working_Age_population,SIMD2020v2_Rank,SIMD_2020v2_Percentile,SIMD2020v2_Vigintile,SIMD2020v2_Decile,SIMD2020v2_Quintile,...,drive_petrol,drive_GP,drive_post,drive_primary,drive_retail,drive_secondary,PT_GP,PT_post,PT_retail,broadband
0,S01006506,Culter,Aberdeen City,894,580,4691,68,14,7,4,...,2.540103,3.074295,1.616239,2.615747,1.544260,9.930833,8.863589,5.856135,6.023406,11%
1,S01006507,Culter,Aberdeen City,793,470,4862,70,14,7,4,...,3.915072,4.309812,2.555858,3.646697,2.849656,11.042816,9.978272,7.515000,7.926029,1%
2,S01006508,Culter,Aberdeen City,624,461,5686,82,17,9,5,...,3.323025,3.784549,1.440991,3.247325,2.062255,10.616768,8.620700,4.321493,5.770910,1%
3,S01006509,Culter,Aberdeen City,537,307,4332,63,13,7,4,...,2.622991,2.778026,2.620681,1.936908,2.160142,10.036471,7.935112,8.433328,8.329819,11%
4,S01006510,Culter,Aberdeen City,663,415,3913,57,12,6,3,...,2.115004,2.358335,2.408416,1.845672,1.784635,9.650000,5.568964,6.966429,6.632609,0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6971,S01013477,Broxburn South,West Lothian,612,355,2464,36,8,4,2,...,2.732500,3.706674,1.690504,3.148465,1.559616,5.930057,7.693930,4.175574,4.284855,0%
6972,S01013478,Broxburn East,West Lothian,817,582,3681,53,11,6,3,...,5.227738,1.922628,1.977860,2.979797,2.815971,3.425031,6.550913,6.485256,7.598948,0%
6973,S01013479,Broxburn East,West Lothian,785,479,1423,21,5,3,2,...,3.664326,3.613521,1.657220,2.530326,1.445423,5.912698,7.355121,4.006667,4.412252,1%
6974,S01013480,Broxburn East,West Lothian,677,410,3291,48,10,5,3,...,3.251885,4.625972,2.600421,3.541645,2.127917,6.626376,10.176170,6.409534,6.928539,0%


In [11]:
Glasgow_SIMD2020 = pd.merge(Glasgow_DataZone, SIMD2020, how = 'inner', left_on='DataZone', right_on='Data_Zone')
Glasgow_SIMD2020 = Glasgow_SIMD2020.dropna(how='any', subset=['income_rate','employment_rate','University'])
Glasgow_SIMD2020['income_rate'] = Glasgow_SIMD2020['income_rate'].str.strip("%").astype('float')/100
Glasgow_SIMD2020['employment_rate'] = Glasgow_SIMD2020['employment_rate'].str.strip("%").astype('float')/100
Glasgow_SIMD2020['University'] = Glasgow_SIMD2020['University'].str.strip("%").astype('float')/100
Glasgow_SIMD2020['income_rate_reverse'] = Glasgow_SIMD2020['income_rate'].map(lambda x: 1 - x)
Glasgow_SIMD2020['employment_rate_reverse'] = Glasgow_SIMD2020['employment_rate'].map(lambda x: 1 - x)

In [12]:
Matrix2020 = Glasgow_SIMD2020.loc[:,['Data_Zone','income_rate_reverse', 'employment_rate_reverse','University']]
Matrix2020 = pd.merge(Matrix2020, houseprice_20, left_on = 'Data_Zone', right_on = 'DataZone', how = 'inner')
Matrix2020 = Matrix2020.reset_index()
Matrix2020_pca = Matrix2020.loc[:,['50%','income_rate_reverse', 'employment_rate_reverse','University']]
Matrix2020_pca

,50%,income_rate_reverse,employment_rate_reverse,University
0,64000.000000,0.97,0.95,0.12
1,54000.000000,0.77,0.82,0.07
2,65000.000000,0.87,0.92,0.15
3,26666.666667,0.75,0.87,0.04
4,74750.000000,0.95,0.96,0.19
...,...,...,...,...
710,21875.000000,0.58,0.69,0.02
711,54250.000000,0.68,0.80,0.07
712,54497.500000,0.77,0.82,0.04
713,55000.000000,0.84,0.87,0.09


In [13]:
pca = decomposition.PCA(n_components=1)             # Only need 1st Principal Component
pca.fit(Matrix2020_pca)                                   #  Train model on data
Matrix2020_T = pd.DataFrame(pca.transform(Matrix2020_pca))  #  Transform data using model

print("The amount of explained variance of the SES score is: {0:6.5f}".format(pca.explained_variance_ratio_[0]))

The amount of explained variance of the SES score is: 1.00000


In [14]:
Matrix2020_T

,0
0,6882.243182
1,-3117.756818
2,7882.243182
3,-30451.090152
4,17632.243182
...,...
710,-35242.756819
711,-2867.756819
712,-2620.256818
713,-2117.756818


In [15]:
Matrix2020_result = pd.concat([Matrix2020,Matrix2020_T],axis = 1)

In [16]:
Matrix2020_result

,index,Data_Zone,income_rate_reverse,employment_rate_reverse,University,DataZone,50%,0
0,0,S01009758,0.97,0.95,0.12,S01009758,64000.000000,6882.243182
1,1,S01009760,0.77,0.82,0.07,S01009760,54000.000000,-3117.756818
2,2,S01009761,0.87,0.92,0.15,S01009761,65000.000000,7882.243182
3,3,S01009762,0.75,0.87,0.04,S01009762,26666.666667,-30451.090152
4,4,S01009763,0.95,0.96,0.19,S01009763,74750.000000,17632.243182
...,...,...,...,...,...,...,...,...
710,710,S01010499,0.58,0.69,0.02,S01010499,21875.000000,-35242.756819
711,711,S01010500,0.68,0.80,0.07,S01010500,54250.000000,-2867.756819
712,712,S01010501,0.77,0.82,0.04,S01010501,54497.500000,-2620.256818
713,713,S01010502,0.84,0.87,0.09,S01010502,55000.000000,-2117.756818


In [17]:
Matrix2016_result.to_csv('E:/文档/Urban Analytics/Dissertation/Data/Score_16.csv')
Matrix2020_result.to_csv('E:/文档/Urban Analytics/Dissertation/Data/Score_20.csv')